In [ ]:
class ArticleCrawling:

    def __init__(self, url):
        self.url = url
        self.soup = self._get_soup()

    def _get_soup(self):
        response = requests.get(self.url)
        return BeautifulSoup(response.text, 'html.parser')
    
    def get_article_url(self):
        meta_tag = self.soup.find("meta", property = "og:url")
        return meta_tag["content"] if meta_tag else None
    
    def get_id(self):
        url = self.get_article_url()    # https://www.spnews.co.kr/news/articleView.html?idxno=101049

        # source(spnew, ytn 등등)
        source = url.split("//")[1]
        source = source.split("/")[0]
        source = source.replace("www.", "").split(".")[0]

        # id
        if "idxno" not in url:
            return None
        idx = url.split("idxno=")[1]
        return f"{source}_{idx}"
    
    def get_title(self):
        header = self.soup.find("h1", class_ = "heading")
        return header.get_text(strip = True) if header else None
      
    def get_author(self):
        tag = self.soup.find("i", class_ = "icon-user-o")
        if tag:
            author = tag.parent.get_text(strip = True)
            return author.replace("기자명", "").strip()
        
    def get_category(self):
        breadcrumb = self.soup.find("ul", class_ = "breadcrumbs")
        if breadcrumb:
            for a in breadcrumb.find_all("a"):
                text = a.get_text(strip=True)
                if text in ["정치", "외교", "군사", "경제/산업", "사회/문화/체육"]:
                    return text
    
    def get_publish_date(self):
        tag = self.soup.find("i", class_ = "icon-clock-o")
        if tag:
            raw = tag.parent.get_text(strip = True)
            return raw.replace("입력", "").strip()
        
    def get_contents(self):
        article_body = self.soup.find_all("span", style = "font-size:18px;")
        contents = "\n".join(i.get_text(strip = True) for i in article_body)
        # return contents.rstrip("@") 
        return contents.split('@')[0].strip()
    
    def to_dict(self):
        return {
        # "ID": ,
        "url": self.get_article_url(),
        "id": self.get_id(),
        "title": self.get_title(),
        "author": self.get_author(),
        "category": self.get_category(),
        "publish_date": self.get_publish_date(),
        "contents": self.get_contents()
            # {
            #     "summary": ,
            #     "vector": ,
            #     "keywords": ,     >> 별개의 db로?
            # }
        }
    
    def __str__(self):
        data = self.to_dict()
        return "\n".join(f"{k}: {v}" for k, v in data.items())
